In [2]:
import os
from urllib.parse import urlparse

def get_domain(line):
    """Extrahuje doménu z první URL v řádku."""
    try:
        # Najde první URL v řádku
        url = line.split()[0]  # předpokládá, že URL je první položka
        parsed_url = urlparse(url)
        return parsed_url.netloc
    except:
        return ''  # pro případ, že není URL nalezena

def process_file(file_path):
    """Zpracuje soubor a seřadí řádky podle domén."""
    # Načtení všech řádků
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    # Seskupení řádků podle domén
    domain_groups = {}
    for line in lines:
        line = line.strip()
        if line:
            domain = get_domain(line)
            if domain not in domain_groups:
                domain_groups[domain] = []
            domain_groups[domain].append(line)
    
    # Sloučení seřazených řádků
    sorted_lines = []
    for domain in sorted(domain_groups.keys()):
        sorted_lines.extend(sorted(domain_groups[domain]))
    
    # Zápis seřazených řádků zpět do souboru
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(sorted_lines))
        f.write('\n')  # přidá prázdný řádek na konec

def process_files_in_directory(directory):
    """Zpracuje všechny soubory v zadaném adresáři."""
    for filename in os.listdir(directory):
        if filename.endswith('_url'):
            file_path = os.path.join(directory, filename)
            print(f"Zpracovávám soubor: {filename}")
            process_file(file_path)
    print("Zpracování dokončeno.")

# Cesta k pracovnímu adresáři
working_directory = '--pracovni/--vysledky/'

# Spuštění zpracování
process_files_in_directory(working_directory)


Zpracovávám soubor: 0-50_url
Zpracovávám soubor: 100-150_url
Zpracovávám soubor: 150-200_url
Zpracovávám soubor: 200-250_url
Zpracovávám soubor: 250-300_url
Zpracovávám soubor: 300-350_url
Zpracovávám soubor: 350-400_url
Zpracovávám soubor: 400-450_url
Zpracovávám soubor: 450-500_url
Zpracovávám soubor: 50-100_url
Zpracovávám soubor: 500-550_url
Zpracovávám soubor: 550-600_url
Zpracovávám soubor: 600-613_url
Zpracování dokončeno.


In [ ]:
import pandas as pd
import requests
from lxml import html
import os
import re
import concurrent.futures
import time
from tqdm import tqdm

# Nastavení cest
base_dir = "--pracovni/--vysledky/"
output_dir = os.path.join(base_dir, "scraped")
os.makedirs(output_dir, exist_ok=True)

# Nastavení hlaviček pro HTTP požadavky
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'cs,en-US;q=0.7,en;q=0.3',
}

# Funkce pro scrapování jedné URL
def scrape_url(url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            tree = html.fromstring(response.content)
            
            # Získání breadcrumbs pomocí XPath
            breadcrumbs = tree.xpath("//span[@itemprop='itemListElement']/a/span[@itemprop='name']/text()")
            breadcrumbs_text = " > ".join(breadcrumbs) if breadcrumbs else ""
            
            # Získání page_type pomocí regex
            page_type = ""
            match = re.search(r'"pageType"\s*:\s*"(.*?)"', response.text)
            if match:
                page_type = match.group(1)
                
            return url, breadcrumbs_text, page_type
        else:
            return url, f"Error: Status code {response.status_code}", ""
    except Exception as e:
        return url, f"Error: {str(e)}", ""

# Funkce pro načtení URL adres z různých formátů souborů
def load_urls_from_file(file_path):
    urls = []
    
    # Zkusíme různé formáty souborů
    try:
        # Zkusíme načíst jako textový soubor (txt, csv)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            # Pokud obsahuje oddělovače, může jít o CSV
            if ',' in content or ';' in content or '\t' in content:
                # Zkusíme různé oddělovače
                for sep in [',', ';', '\t']:
                    try:
                        df = pd.read_csv(file_path, sep=sep, encoding='utf-8')
                        if not df.empty:
                            urls = df.iloc[:, 0].tolist()  # Bereme první sloupec
                            print(f"Soubor úspěšně načten jako CSV s oddělovačem '{sep}'")
                            return urls
                    except:
                        continue
            
            # Pokud to není CSV, zkusíme to jako prostý textový soubor s URL na každém řádku
            urls = [line.strip() for line in content.split('\n') if line.strip()]
            if urls:
                print("Soubor úspěšně načten jako textový soubor")
                return urls
    except:
        pass
    
    # Zkusíme načíst jako Excel
    try:
        df = pd.read_excel(file_path)
        urls = df.iloc[:, 0].tolist()
        print("Soubor úspěšně načten jako Excel")
        return urls
    except:
        pass
    
    # Pokud nic nefunguje, vyhodíme výjimku
    raise Exception("Nepodařilo se rozpoznat formát souboru")

# Funkce pro zpracování jednoho souboru
def process_file(file_name):
    print(f"Zpracovávám soubor: {file_name}")
    file_path = os.path.join(base_dir, file_name)
    output_file = os.path.join(output_dir, file_name + ".csv")
    
    # Načtení URL adres ze souboru
    try:
        urls = load_urls_from_file(file_path)
        if not urls:
            print(f"Soubor {file_name} neobsahuje žádné URL adresy")
            return False
    except Exception as e:
        print(f"Chyba při načítání souboru {file_name}: {str(e)}")
        return False
    
    results = []
    
    # Paralelní zpracování URL adres
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(scrape_url, url): url for url in urls}
        
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc=f"Scrapování {file_name}"):
            url, breadcrumbs, page_type = future.result()
            results.append((url, breadcrumbs, page_type))
    
    # Vytvoření DataFrame a uložení výsledků
    result_df = pd.DataFrame(results, columns=['URL', 'Breadcrumbs', 'PageType'])
    result_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Výsledky uloženy do: {output_file}")
    return True

# Hlavní funkce pro zpracování všech souborů
def process_all_files():
    file_ranges = [
        (50, 100), (100, 150), (150, 200), (200, 250),
        (250, 300), (300, 350), (350, 400), (400, 450), (450, 500),
        (500, 550), (550, 600), (600, 613)
    ]
    
    for start, end in file_ranges:
        file_name = f"{start}-{end}_url"
        success = process_file(file_name)
        if not success:
            print(f"Přeskakuji soubor {file_name} kvůli chybě.")
        time.sleep(1)  # Krátká pauza mezi soubory

# Spuštění hlavního procesu
if __name__ == "__main__":
    print("Začínám scrapování URL adres...")
    process_all_files()
    print("Scrapování dokončeno!")

Začínám scrapování URL adres...
Zpracovávám soubor: 0-50_url
Soubor úspěšně načten jako textový soubor


Scrapování 0-50_url: 100%|██████████| 113769/113769 [4:59:39<00:00,  6.33it/s]     


Výsledky uloženy do: --pracovni/--vysledky/scraped\0-50_url.csv
Zpracovávám soubor: 50-100_url
Soubor úspěšně načten jako textový soubor


Scrapování 50-100_url:   0%|          | 121/206475 [00:03<1:50:36, 31.09it/s]


In [30]:
with open('--pracovni/--vysledky/50-100_url', 'r') as file:
    first_lines = [file.readline().strip() for _ in range(5)]
    print(first_lines)


FileNotFoundError: [Errno 2] No such file or directory: '--pracovni/--vysledky/50-100_url'

In [11]:
import pandas as pd
import requests
from lxml import html
import os
import re
import concurrent.futures
import time
from tqdm import tqdm

# Nastavení cest
base_dir = "1pracovni/1vysledky/"
output_dir = os.path.join(base_dir, "scraped")
os.makedirs(output_dir, exist_ok=True)

# Nastavení hlaviček pro HTTP požadavky
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'cs,en-US;q=0.7,en;q=0.3',
}

# Funkce pro scrapování jedné URL
def scrape_url(url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            tree = html.fromstring(response.content)

            # Získání breadcrumbs pomocí XPath
            breadcrumbs = tree.xpath("//span[@itemprop='itemListElement']/a/span[@itemprop='name']/text()")
            breadcrumbs_text = " > ".join(breadcrumbs) if breadcrumbs else ""

            # Získání page_type pomocí regex
            page_type = ""
            match = re.search(r'"pageType"\s*:\s*"(.*?)"', response.text)
            if match:
                page_type = match.group(1)

            return url, breadcrumbs_text, page_type
        else:
            return url, f"Error: Status code {response.status_code}", ""
    except Exception as e:
        return url, f"Error: {str(e)}", ""

# Funkce pro načtení URL adres z různých formátů souborů
def load_urls_from_file(file_path):
    urls = []

    # Zkusíme různé formáty souborů
    try:
        # Zkusíme načíst jako textový soubor (txt, csv)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            # Pokud obsahuje oddělovače, může jít o CSV
            if ',' in content or ';' in content or '\t' in content:
                # Zkusíme různé oddělovače
                for sep in [',', ';', '\t']:
                    try:
                        df = pd.read_csv(file_path, sep=sep, encoding='utf-8')
                        if not df.empty:
                            urls = df.iloc[:, 0].tolist()  # Bereme první sloupec
                            print(f"Soubor úspěšně načten jako CSV s oddělovačem '{sep}'")
                            return urls
                    except:
                        continue

            # Pokud to není CSV, zkusíme to jako prostý textový soubor s URL na každém řádku
            urls = [line.strip() for line in content.split('\n') if line.strip()]
            if urls:
                print("Soubor úspěšně načten jako textový soubor")
                return urls
    except:
        pass

    # Zkusíme načíst jako Excel
    try:
        df = pd.read_excel(file_path)
        urls = df.iloc[:, 0].tolist()
        print("Soubor úspěšně načten jako Excel")
        return urls
    except:
        pass

    # Pokud nic nefunguje, vyhodíme výjimku
    raise Exception("Nepodařilo se rozpoznat formát souboru")

# Funkce pro zpracování jednoho souboru
def process_file(file_name):
    print(f"Zpracovávám soubor: {file_name}")
    file_path = os.path.join(base_dir, file_name)
    output_file = os.path.join(output_dir, file_name + ".csv")

    # Načtení URL adres ze souboru
    try:
        urls = load_urls_from_file(file_path)
        if not urls:
            print(f"Soubor {file_name} neobsahuje žádné URL adresy")
            return False
    except Exception as e:
        print(f"Chyba při načítání souboru {file_name}: {str(e)}")
        return False

    results = []

    # Paralelní zpracování URL adres
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(scrape_url, url): url for url in urls}

        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc=f"Scrapování {file_name}"):
            url, breadcrumbs, page_type = future.result()
            results.append((url, breadcrumbs, page_type))

    # Vytvoření DataFrame a uložení výsledků
    result_df = pd.DataFrame(results, columns=['URL', 'Breadcrumbs', 'PageType'])
    result_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Výsledky uloženy do: {output_file}")
    return True

# Hlavní funkce pro zpracování všech souborů
def process_all_files():
    file_ranges = [
        (100, 150), (150, 200), (200, 250),
        (250, 300), (300, 350), (350, 400), (400, 450), (450, 500),
        (500, 550), (550, 600), (600, 613)
    ]

    for start, end in file_ranges:
        file_name = f"{start}-{end}_url"
        success = process_file(file_name)
        if not success:
            print(f"Přeskakuji soubor {file_name} kvůli chybě.")
        time.sleep(1)  # Krátká pauza mezi soubory

# Spuštění hlavního procesu
if __name__ == "__main__":
    print("Začínám scrapování URL adres...")
    process_all_files()
    print("Scrapování dokončeno!")


Začínám scrapování URL adres...
Zpracovávám soubor: 100-150_url
Chyba při načítání souboru 100-150_url: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 100-150_url kvůli chybě.
Zpracovávám soubor: 150-200_url
Chyba při načítání souboru 150-200_url: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 150-200_url kvůli chybě.
Zpracovávám soubor: 200-250_url
Chyba při načítání souboru 200-250_url: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 200-250_url kvůli chybě.
Zpracovávám soubor: 250-300_url
Chyba při načítání souboru 250-300_url: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 250-300_url kvůli chybě.
Zpracovávám soubor: 300-350_url
Chyba při načítání souboru 300-350_url: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 300-350_url kvůli chybě.
Zpracovávám soubor: 350-400_url
Chyba při načítání souboru 350-400_url: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 350-400_url kvůli chybě.
Zpracovávám soubor: 400-450_url
Chyba při na

KeyboardInterrupt: 

In [13]:
import pandas as pd
import requests
from lxml import html
import os
import re
import concurrent.futures
import time
from tqdm import tqdm

# Nastavení cest
base_dir = "1pracovni/1vysledky/"
output_dir = os.path.join(base_dir, "scraped")
os.makedirs(output_dir, exist_ok=True)

# Nastavení hlaviček pro HTTP požadavky
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'cs,en-US;q=0.7,en;q=0.3',
}

# Funkce pro scrapování jedné URL
def scrape_url(url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            tree = html.fromstring(response.content)

            # Získání breadcrumbs pomocí XPath
            breadcrumbs = tree.xpath("//span[@itemprop='itemListElement']/a/span[@itemprop='name']/text()")
            breadcrumbs_text = " > ".join(breadcrumbs) if breadcrumbs else ""

            # Získání page_type pomocí regex
            page_type = ""
            match = re.search(r'"pageType"\s*:\s*"(.*?)"', response.text)
            if match:
                page_type = match.group(1)

            return url, breadcrumbs_text, page_type
        else:
            return url, f"Error: Status code {response.status_code}", ""
    except Exception as e:
        return url, f"Error: {str(e)}", ""

# Funkce pro načtení URL adres z CSV souboru
def load_urls_from_file(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
        if not df.empty:
            urls = df.iloc[:, 0].tolist()  # Bereme první sloupec
            print(f"Soubor úspěšně načten jako CSV")
            return urls
    except Exception as e:
        raise Exception(f"Nepodařilo se načíst soubor: {str(e)}")

# Funkce pro zpracování jednoho souboru
def process_file(file_name):
    print(f"Zpracovávám soubor: {file_name}")
    file_path = os.path.join(base_dir, file_name)
    output_file = os.path.join(output_dir, file_name)

    # Načtení URL adres ze souboru
    try:
        urls = load_urls_from_file(file_path)
        if not urls:
            print(f"Soubor {file_name} neobsahuje žádné URL adresy")
            return False
    except Exception as e:
        print(f"Chyba při načítání souboru {file_name}: {str(e)}")
        return False

    results = []

    # Paralelní zpracování URL adres
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(scrape_url, url): url for url in urls}

        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc=f"Scrapování {file_name}"):
            url, breadcrumbs, page_type = future.result()
            results.append((url, breadcrumbs, page_type))

    # Vytvoření DataFrame a uložení výsledků
    result_df = pd.DataFrame(results, columns=['URL', 'Breadcrumbs', 'PageType'])
    result_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Výsledky uloženy do: {output_file}")
    return True

# Hlavní funkce pro zpracování všech souborů
def process_all_files():
    file_ranges = [
        (100, 150), (150, 200), (200, 250),
        (250, 300), (300, 350), (350, 400), (400, 450), (450, 500),
        (500, 550), (550, 600), (600, 613)
    ]

    for start, end in file_ranges:
        file_name = f"{start}-{end}_url.csv"
        success = process_file(file_name)
        if not success:
            print(f"Přeskakuji soubor {file_name} kvůli chybě.")
        time.sleep(1)  # Krátká pauza mezi soubory

# Spuštění hlavního procesu
if __name__ == "__main__":
    print("Začínám scrapování URL adres...")
    process_all_files()
    print("Scrapování dokončeno!")


Začínám scrapování URL adres...
Zpracovávám soubor: 100-150_url.csv
Chyba při načítání souboru 100-150_url.csv: Nepodařilo se načíst soubor: [Errno 2] No such file or directory: '1pracovni/1vysledky/100-150_url.csv'
Přeskakuji soubor 100-150_url.csv kvůli chybě.
Zpracovávám soubor: 150-200_url.csv
Chyba při načítání souboru 150-200_url.csv: Nepodařilo se načíst soubor: [Errno 2] No such file or directory: '1pracovni/1vysledky/150-200_url.csv'
Přeskakuji soubor 150-200_url.csv kvůli chybě.
Zpracovávám soubor: 200-250_url.csv
Chyba při načítání souboru 200-250_url.csv: Nepodařilo se načíst soubor: [Errno 2] No such file or directory: '1pracovni/1vysledky/200-250_url.csv'
Přeskakuji soubor 200-250_url.csv kvůli chybě.
Zpracovávám soubor: 250-300_url.csv
Chyba při načítání souboru 250-300_url.csv: Nepodařilo se načíst soubor: [Errno 2] No such file or directory: '1pracovni/1vysledky/250-300_url.csv'
Přeskakuji soubor 250-300_url.csv kvůli chybě.
Zpracovávám soubor: 300-350_url.csv
Chyba př

In [15]:
import pandas as pd
import requests
from lxml import html
import os
import re
import concurrent.futures
import time
from tqdm import tqdm

# Nastavení cest
base_dir = "1pracovni/1vysledky/"
output_dir = os.path.join(base_dir, "scraped")
os.makedirs(output_dir, exist_ok=True)

# Nastavení hlaviček pro HTTP požadavky
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'cs,en-US;q=0.7,en;q=0.3',
}

# Funkce pro scrapování jedné URL
def scrape_url(url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            tree = html.fromstring(response.content)

            # Získání breadcrumbs pomocí XPath
            breadcrumbs = tree.xpath("//span[@itemprop='itemListElement']/a/span[@itemprop='name']/text()")
            breadcrumbs_text = " > ".join(breadcrumbs) if breadcrumbs else ""

            # Získání page_type pomocí regex
            page_type = ""
            match = re.search(r'"pageType"\s*:\s*"(.*?)"', response.text)
            if match:
                page_type = match.group(1)

            return url, breadcrumbs_text, page_type
        else:
            return url, f"Error: Status code {response.status_code}", ""
    except Exception as e:
        return url, f"Error: {str(e)}", ""

# Funkce pro načtení URL adres z CSV souboru
def load_urls_from_file(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
        if not df.empty:
            urls = df.iloc[:, 0].tolist()  # Bereme první sloupec
            print(f"Soubor úspěšně načten jako CSV")
            return urls
    except Exception as e:
        raise Exception(f"Nepodařilo se načíst soubor: {str(e)}")

# Funkce pro zpracování jednoho souboru
def process_file(file_name):
    print(f"Zpracovávám soubor: {file_name}")
    file_path = os.path.join(base_dir, file_name)
    output_file = os.path.join(output_dir, file_name)

    # Check if the file exists
    if not os.path.exists(file_path):
        print(f"Soubor {file_name} neexistuje.")
        return False

    # Načtení URL adres ze souboru
    try:
        urls = load_urls_from_file(file_path)
        if not urls:
            print(f"Soubor {file_name} neobsahuje žádné URL adresy")
            return False
    except Exception as e:
        print(f"Chyba při načítání souboru {file_name}: {str(e)}")
        return False

    results = []

    # Paralelní zpracování URL adres
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(scrape_url, url): url for url in urls}

        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc=f"Scrapování {file_name}"):
            url, breadcrumbs, page_type = future.result()
            results.append((url, breadcrumbs, page_type))

    # Vytvoření DataFrame a uložení výsledků
    result_df = pd.DataFrame(results, columns=['URL', 'Breadcrumbs', 'PageType'])
    result_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Výsledky uloženy do: {output_file}")
    return True

# Hlavní funkce pro zpracování všech souborů
def process_all_files():
    file_ranges = [
        (100, 150), (150, 200), (200, 250),
        (250, 300), (300, 350), (350, 400), (400, 450), (450, 500),
        (500, 550), (550, 600), (600, 613)
    ]

    for start, end in file_ranges:
        file_name = f"{start}-{end}_url.csv"
        success = process_file(file_name)
        if not success:
            print(f"Přeskakuji soubor {file_name} kvůli chybě.")
        time.sleep(1)  # Krátká pauza mezi soubory

# Spuštění hlavního procesu
if __name__ == "__main__":
    print("Začínám scrapování URL adres...")
    process_all_files()
    print("Scrapování dokončeno!")


Začínám scrapování URL adres...
Zpracovávám soubor: 100-150_url.csv
Soubor 100-150_url.csv neexistuje.
Přeskakuji soubor 100-150_url.csv kvůli chybě.
Zpracovávám soubor: 150-200_url.csv
Soubor 150-200_url.csv neexistuje.
Přeskakuji soubor 150-200_url.csv kvůli chybě.
Zpracovávám soubor: 200-250_url.csv
Soubor 200-250_url.csv neexistuje.
Přeskakuji soubor 200-250_url.csv kvůli chybě.
Zpracovávám soubor: 250-300_url.csv
Soubor 250-300_url.csv neexistuje.
Přeskakuji soubor 250-300_url.csv kvůli chybě.
Zpracovávám soubor: 300-350_url.csv
Soubor 300-350_url.csv neexistuje.
Přeskakuji soubor 300-350_url.csv kvůli chybě.


KeyboardInterrupt: 

In [17]:
import pandas as pd
import requests
from lxml import html
import os
import re
import concurrent.futures
import time
from tqdm import tqdm

# Nastavení cest
base_dir = "--pracovni/--vysledky/"
output_dir = os.path.join(base_dir, "scraped")
os.makedirs(output_dir, exist_ok=True)

# Nastavení hlaviček pro HTTP požadavky
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'cs,en-US;q=0.7,en;q=0.3',
}

# Funkce pro scrapování jedné URL
def scrape_url(url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            tree = html.fromstring(response.content)
            
            # Získání breadcrumbs pomocí XPath
            breadcrumbs = tree.xpath("//span[@itemprop='itemListElement']/a/span[@itemprop='name']/text()")
            breadcrumbs_text = " > ".join(breadcrumbs) if breadcrumbs else ""
            
            # Získání page_type pomocí regex
            page_type = ""
            match = re.search(r'"pageType"\s*:\s*"(.*?)"', response.text)
            if match:
                page_type = match.group(1)
                
            return url, breadcrumbs_text, page_type
        else:
            return url, f"Error: Status code {response.status_code}", ""
    except Exception as e:
        return url, f"Error: {str(e)}", ""

# Funkce pro načtení URL adres z různých formátů souborů
def load_urls_from_file(file_path):
    urls = []
    
    # Zkusíme různé formáty souborů
    try:
        # Zkusíme načíst jako textový soubor (txt, csv)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            # Pokud obsahuje oddělovače, může jít o CSV
            if ',' in content or ';' in content or '\t' in content:
                # Zkusíme různé oddělovače
                for sep in [',', ';', '\t']:
                    try:
                        df = pd.read_csv(file_path, sep=sep, encoding='utf-8')
                        if not df.empty:
                            urls = df.iloc[:, 0].tolist()  # Bereme první sloupec
                            print(f"Soubor úspěšně načten jako CSV s oddělovačem '{sep}'")
                            return urls
                    except:
                        continue
            
            # Pokud to není CSV, zkusíme to jako prostý textový soubor s URL na každém řádku
            urls = [line.strip() for line in content.split('\n') if line.strip()]
            if urls:
                print("Soubor úspěšně načten jako textový soubor")
                return urls
    except:
        pass
    
    # Zkusíme načíst jako Excel
    try:
        df = pd.read_excel(file_path)
        urls = df.iloc[:, 0].tolist()
        print("Soubor úspěšně načten jako Excel")
        return urls
    except:
        pass
    
    # Pokud nic nefunguje, vyhodíme výjimku
    raise Exception("Nepodařilo se rozpoznat formát souboru")

# Funkce pro zpracování jednoho souboru
def process_file(file_name):
    print(f"Zpracovávám soubor: {file_name}")
    file_path = os.path.join(base_dir, file_name)
    output_file = os.path.join(output_dir, file_name + ".csv")
    
    # Načtení URL adres ze souboru
    try:
        urls = load_urls_from_file(file_path)
        if not urls:
            print(f"Soubor {file_name} neobsahuje žádné URL adresy")
            return False
    except Exception as e:
        print(f"Chyba při načítání souboru {file_name}: {str(e)}")
        return False
    
    results = []
    
    # Paralelní zpracování URL adres
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(scrape_url, url): url for url in urls}
        
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc=f"Scrapování {file_name}"):
            url, breadcrumbs, page_type = future.result()
            results.append((url, breadcrumbs, page_type))
    
    # Vytvoření DataFrame a uložení výsledků
    result_df = pd.DataFrame(results, columns=['URL', 'Breadcrumbs', 'PageType'])
    result_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Výsledky uloženy do: {output_file}")
    return True

# Hlavní funkce pro zpracování všech souborů
def process_all_files():
    file_ranges = [
        (50, 100), (100, 150), (150, 200), (200, 250),
        (250, 300), (300, 350), (350, 400), (400, 450), (450, 500),
        (500, 550), (550, 600), (600, 613)
    ]
    
    for start, end in file_ranges:
        file_name = f"{start}-{end}_url"
        success = process_file(file_name)
        if not success:
            print(f"Přeskakuji soubor {file_name} kvůli chybě.")
        time.sleep(1)  # Krátká pauza mezi soubory

# Spuštění hlavního procesu
if __name__ == "__main__":
    print("Začínám scrapování URL adres...")
    process_all_files()
    print("Scrapování dokončeno!")

Začínám scrapování URL adres...
Zpracovávám soubor: 50-100_url
Chyba při načítání souboru 50-100_url: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 50-100_url kvůli chybě.
Zpracovávám soubor: 100-150_url
Chyba při načítání souboru 100-150_url: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 100-150_url kvůli chybě.
Zpracovávám soubor: 150-200_url
Chyba při načítání souboru 150-200_url: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 150-200_url kvůli chybě.
Zpracovávám soubor: 200-250_url
Chyba při načítání souboru 200-250_url: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 200-250_url kvůli chybě.
Zpracovávám soubor: 250-300_url
Chyba při načítání souboru 250-300_url: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 250-300_url kvůli chybě.
Zpracovávám soubor: 300-350_url
Chyba při načítání souboru 300-350_url: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 300-350_url kvůli chybě.
Zpracovávám soubor: 350-400_url
Chyba při načít

In [19]:
import pandas as pd
import requests
from lxml import html
import os
import re
import concurrent.futures
import time
from tqdm import tqdm

# Nastavení cest
base_dir = "1pracovni/1vysledky/"
output_dir = os.path.join(base_dir, "scraped")
os.makedirs(output_dir, exist_ok=True)

# Nastavení hlaviček pro HTTP požadavky
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'cs,en-US;q=0.7,en;q=0.3',
}

# Funkce pro scrapování jedné URL
def scrape_url(url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            tree = html.fromstring(response.content)

            # Získání breadcrumbs pomocí XPath
            breadcrumbs = tree.xpath("//span[@itemprop='itemListElement']/a/span[@itemprop='name']/text()")
            breadcrumbs_text = " > ".join(breadcrumbs) if breadcrumbs else ""

            # Získání page_type pomocí regex
            page_type = ""
            match = re.search(r'"pageType"\s*:\s*"(.*?)"', response.text)
            if match:
                page_type = match.group(1)

            return url, breadcrumbs_text, page_type
        else:
            return url, f"Error: Status code {response.status_code}", ""
    except Exception as e:
        return url, f"Error: {str(e)}", ""

# Funkce pro načtení URL adres z různých formátů souborů
def load_urls_from_file(file_path):
    urls = []

    # Zkusíme různé formáty souborů
    try:
        # Zkusíme načíst jako textový soubor (txt, csv)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            # Pokud obsahuje oddělovače, může jít o CSV
            if ',' in content or ';' in content or '\t' in content:
                # Zkusíme různé oddělovače
                for sep in [',', ';', '\t']:
                    try:
                        df = pd.read_csv(file_path, sep=sep, encoding='utf-8', header=None)
                        if not df.empty:
                            urls = df.iloc[:, 0].tolist()  # Bereme první sloupec
                            print(f"Soubor úspěšně načten jako CSV s oddělovačem '{sep}'")
                            return urls
                    except:
                        continue

            # Pokud to není CSV, zkusíme to jako prostý textový soubor s URL na každém řádku
            urls = [line.strip() for line in content.split('\n') if line.strip()]
            if urls:
                print("Soubor úspěšně načten jako textový soubor")
                return urls
    except:
        pass

    # Zkusíme načíst jako Excel
    try:
        df = pd.read_excel(file_path, header=None)
        urls = df.iloc[:, 0].tolist()
        print("Soubor úspěšně načten jako Excel")
        return urls
    except:
        pass

    # Pokud nic nefunguje, vyhodíme výjimku
    raise Exception("Nepodařilo se rozpoznat formát souboru")

# Funkce pro zpracování jednoho souboru
def process_file(file_name):
    print(f"Zpracovávám soubor: {file_name}")
    file_path = os.path.join(base_dir, file_name)
    output_file = os.path.join(output_dir, file_name + ".csv")

    # Načtení URL adres ze souboru
    try:
        urls = load_urls_from_file(file_path)
        if not urls:
            print(f"Soubor {file_name} neobsahuje žádné URL adresy")
            return False
    except Exception as e:
        print(f"Chyba při načítání souboru {file_name}: {str(e)}")
        return False

    results = []

    # Paralelní zpracování URL adres
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(scrape_url, url): url for url in urls}

        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc=f"Scrapování {file_name}"):
            url, breadcrumbs, page_type = future.result()
            results.append((url, breadcrumbs, page_type))

    # Vytvoření DataFrame a uložení výsledků
    result_df = pd.DataFrame(results, columns=['URL', 'Breadcrumbs', 'PageType'])
    result_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Výsledky uloženy do: {output_file}")
    return True

# Hlavní funkce pro zpracování všech souborů
def process_all_files():
    file_ranges = [
        (50, 100), (100, 150), (150, 200), (200, 250),
        (250, 300), (300, 350), (350, 400), (400, 450), (450, 500),
        (500, 550), (550, 600), (600, 613)
    ]

    for start, end in file_ranges:
        file_name = f"{start}-{end}_url.csv"
        success = process_file(file_name)
        if not success:
            print(f"Přeskakuji soubor {file_name} kvůli chybě.")
        time.sleep(1)  # Krátká pauza mezi soubory

# Spuštění hlavního procesu
if __name__ == "__main__":
    print("Začínám scrapování URL adres...")
    process_all_files()
    print("Scrapování dokončeno!")


Začínám scrapování URL adres...
Zpracovávám soubor: 50-100_url.csv
Chyba při načítání souboru 50-100_url.csv: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 50-100_url.csv kvůli chybě.
Zpracovávám soubor: 100-150_url.csv
Chyba při načítání souboru 100-150_url.csv: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 100-150_url.csv kvůli chybě.
Zpracovávám soubor: 150-200_url.csv
Chyba při načítání souboru 150-200_url.csv: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 150-200_url.csv kvůli chybě.
Zpracovávám soubor: 200-250_url.csv
Chyba při načítání souboru 200-250_url.csv: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 200-250_url.csv kvůli chybě.
Zpracovávám soubor: 250-300_url.csv
Chyba při načítání souboru 250-300_url.csv: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 250-300_url.csv kvůli chybě.
Zpracovávám soubor: 300-350_url.csv
Chyba při načítání souboru 300-350_url.csv: Nepodařilo se rozpoznat formát souboru
Přeskakuji soubor 300-

In [21]:
import os
import pandas as pd

# Nastavení cesty k adresáři s vstupními soubory
input_dir = "1pracovni/1vysledky/"

# Funkce pro analýzu souboru
def analyze_file(file_path):
    try:
        # Zkusíme načíst jako textový soubor
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            if content.strip():
                print(f"Soubor {file_path} je textový soubor.")
                return

        # Zkusíme načíst jako CSV
        for sep in [',', ';', '\t']:
            try:
                df = pd.read_csv(file_path, sep=sep, encoding='utf-8', header=None)
                if not df.empty:
                    print(f"Soubor {file_path} je CSV s oddělovačem '{sep}'.")
                    print("Obsah souboru:")
                    print(df.head())
                    return
            except:
                continue

        # Zkusíme načíst jako Excel
        try:
            df = pd.read_excel(file_path, header=None)
            if not df.empty:
                print(f"Soubor {file_path} je Excel soubor.")
                print("Obsah souboru:")
                print(df.head())
                return
        except:
            pass

        print(f"Soubor {file_path} má neznámý formát nebo je prázdný.")

    except Exception as e:
        print(f"Chyba při analýze souboru {file_path}: {str(e)}")

# Funkce pro analýzu všech souborů v adresáři
def analyze_all_files():
    for file_name in os.listdir(input_dir):
        if file_name.endswith(".csv") or file_name.endswith(".txt") or file_name.endswith(".xlsx"):
            file_path = os.path.join(input_dir, file_name)
            analyze_file(file_path)

# Spuštění analýzy
if __name__ == "__main__":
    print("Začínám analýzu souborů...")
    analyze_all_files()
    print("Analýza dokončena!")


Začínám analýzu souborů...
Analýza dokončena!


In [23]:
import os
import pandas as pd

# Nastavení cesty k adresáři s vstupními soubory
input_dir = "1pracovni/1vysledky/"

# Funkce pro analýzu souboru
def analyze_file(file_path):
    try:
        # Zkusíme načíst jako textový soubor
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            if content.strip():
                print(f"Soubor {file_path} je textový soubor.")
                print("Ukázka obsahu:")
                print(content[:500])  # Vypíšeme prvních 500 znaků
                return

        # Zkusíme načíst jako CSV
        for sep in [',', ';', '\t']:
            try:
                df = pd.read_csv(file_path, sep=sep, encoding='utf-8', header=None)
                if not df.empty:
                    print(f"Soubor {file_path} je CSV s oddělovačem '{sep}'.")
                    print("Obsah souboru:")
                    print(df.head())
                    return
            except:
                continue

        # Zkusíme načíst jako Excel
        try:
            df = pd.read_excel(file_path, header=None)
            if not df.empty:
                print(f"Soubor {file_path} je Excel soubor.")
                print("Obsah souboru:")
                print(df.head())
                return
        except:
            pass

        print(f"Soubor {file_path} má neznámý formát nebo je prázdný.")

    except Exception as e:
        print(f"Chyba při analýze souboru {file_path}: {str(e)}")

# Funkce pro analýzu všech souborů v adresáři
def analyze_all_files():
    for file_name in os.listdir(input_dir):
        if file_name.endswith(".csv") or file_name.endswith(".txt") or file_name.endswith(".xlsx"):
            file_path = os.path.join(input_dir, file_name)
            analyze_file(file_path)

# Spuštění analýzy
if __name__ == "__main__":
    print("Začínám analýzu souborů...")
    analyze_all_files()
    print("Analýza dokončena!")


Začínám analýzu souborů...
Analýza dokončena!


In [25]:
import pandas as pd
import os

# Nastavení cest
base_dir = "1pracovni/1vysledky/"

# Funkce pro ověření validity vstupního souboru
def validate_input_file(file_path):
    urls = []

    # Zkusíme různé formáty souborů
    try:
        # Zkusíme načíst jako textový soubor (txt, csv)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            # Pokud obsahuje oddělovače, může jít o CSV
            if ',' in content or ';' in content or '\t' in content:
                # Zkusíme různé oddělovače
                for sep in [',', ';', '\t']:
                    try:
                        df = pd.read_csv(file_path, sep=sep, encoding='utf-8')
                        if not df.empty:
                            urls = df.iloc[:, 0].tolist()  # Bereme první sloupec
                            print(f"Soubor úspěšně načten jako CSV s oddělovačem '{sep}'")
                            return True
                    except:
                        continue

            # Pokud to není CSV, zkusíme to jako prostý textový soubor s URL na každém řádku
            urls = [line.strip() for line in content.split('\n') if line.strip()]
            if urls:
                print("Soubor úspěšně načten jako textový soubor")
                return True
    except:
        pass

    # Zkusíme načíst jako Excel
    try:
        df = pd.read_excel(file_path)
        urls = df.iloc[:, 0].tolist()
        print("Soubor úspěšně načten jako Excel")
        return True
    except:
        pass

    # Pokud nic nefunguje, soubor není validní
    return False

# Funkce pro zpracování jednoho souboru
def validate_file(file_name):
    print(f"Ověřuji soubor: {file_name}")
    file_path = os.path.join(base_dir, file_name)

    # Ověření validity souboru
    if not validate_input_file(file_path):
        print(f"Soubor {file_name} není validní")
        return False

    print(f"Soubor {file_name} je validní")
    return True

# Hlavní funkce pro ověření všech souborů
def validate_all_files():
    file_ranges = [
        (100, 150), (150, 200), (200, 250),
        (250, 300), (300, 350), (350, 400), (400, 450), (450, 500),
        (500, 550), (550, 600), (600, 613)
    ]

    for start, end in file_ranges:
        file_name = f"{start}-{end}_url.csv"
        validate_file(file_name)

# Spuštění hlavního procesu
if __name__ == "__main__":
    print("Začínám ověřování vstupních souborů...")
    validate_all_files()
    print("Ověřování dokončeno!")


Začínám ověřování vstupních souborů...
Ověřuji soubor: 100-150_url.csv
Soubor 100-150_url.csv není validní
Ověřuji soubor: 150-200_url.csv
Soubor 150-200_url.csv není validní
Ověřuji soubor: 200-250_url.csv
Soubor 200-250_url.csv není validní
Ověřuji soubor: 250-300_url.csv
Soubor 250-300_url.csv není validní
Ověřuji soubor: 300-350_url.csv
Soubor 300-350_url.csv není validní
Ověřuji soubor: 350-400_url.csv
Soubor 350-400_url.csv není validní
Ověřuji soubor: 400-450_url.csv
Soubor 400-450_url.csv není validní
Ověřuji soubor: 450-500_url.csv
Soubor 450-500_url.csv není validní
Ověřuji soubor: 500-550_url.csv
Soubor 500-550_url.csv není validní
Ověřuji soubor: 550-600_url.csv
Soubor 550-600_url.csv není validní
Ověřuji soubor: 600-613_url.csv
Soubor 600-613_url.csv není validní
Ověřování dokončeno!


In [27]:
import pandas as pd
import os

# Nastavení cest
base_dir = "1pracovni/1vysledky/"

# Funkce pro ověření validity vstupního souboru
def validate_input_file(file_path):
    urls = []

    # Zkusíme různé formáty souborů
    try:
        # Zkusíme načíst jako textový soubor (txt, csv)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            # Pokud obsahuje oddělovače, může jít o CSV
            if ',' in content or ';' in content or '\t' in content:
                # Zkusíme různé oddělovače
                for sep in [',', ';', '\t']:
                    try:
                        df = pd.read_csv(file_path, sep=sep, encoding='utf-8')
                        if not df.empty:
                            urls = df.iloc[:, 0].tolist()  # Bereme první sloupec
                            print(f"Soubor úspěšně načten jako CSV s oddělovačem '{sep}'")
                            return True
                    except Exception as e:
                        print(f"Chyba při načítání CSV s oddělovačem '{sep}': {e}")
                        continue

            # Pokud to není CSV, zkusíme to jako prostý textový soubor s URL na každém řádku
            urls = [line.strip() for line in content.split('\n') if line.strip()]
            if urls:
                print("Soubor úspěšně načten jako textový soubor")
                return True
    except Exception as e:
        print(f"Chyba při čtení textového souboru: {e}")
        pass

    # Zkusíme načíst jako Excel
    try:
        df = pd.read_excel(file_path)
        urls = df.iloc[:, 0].tolist()
        print("Soubor úspěšně načten jako Excel")
        return True
    except Exception as e:
        print(f"Chyba při načítání Excel souboru: {e}")
        pass

    # Pokud nic nefunguje, soubor není validní
    return False

# Funkce pro zpracování jednoho souboru
def validate_file(file_name):
    print(f"Ověřuji soubor: {file_name}")
    file_path = os.path.join(base_dir, file_name)

    # Ověření existence souboru
    if not os.path.exists(file_path):
        print(f"Soubor {file_name} neexistuje")
        return False

    # Ověření validity souboru
    if not validate_input_file(file_path):
        print(f"Soubor {file_name} není validní")
        return False

    print(f"Soubor {file_name} je validní")
    return True

# Hlavní funkce pro ověření všech souborů
def validate_all_files():
    file_ranges = [
        (100, 150), (150, 200), (200, 250),
        (250, 300), (300, 350), (350, 400), (400, 450), (450, 500),
        (500, 550), (550, 600), (600, 613)
    ]

    for start, end in file_ranges:
        file_name = f"{start}-{end}_url.csv"
        validate_file(file_name)

# Spuštění hlavního procesu
if __name__ == "__main__":
    print("Začínám ověřování vstupních souborů...")
    validate_all_files()
    print("Ověřování dokončeno!")


Začínám ověřování vstupních souborů...
Ověřuji soubor: 100-150_url.csv
Soubor 100-150_url.csv neexistuje
Ověřuji soubor: 150-200_url.csv
Soubor 150-200_url.csv neexistuje
Ověřuji soubor: 200-250_url.csv
Soubor 200-250_url.csv neexistuje
Ověřuji soubor: 250-300_url.csv
Soubor 250-300_url.csv neexistuje
Ověřuji soubor: 300-350_url.csv
Soubor 300-350_url.csv neexistuje
Ověřuji soubor: 350-400_url.csv
Soubor 350-400_url.csv neexistuje
Ověřuji soubor: 400-450_url.csv
Soubor 400-450_url.csv neexistuje
Ověřuji soubor: 450-500_url.csv
Soubor 450-500_url.csv neexistuje
Ověřuji soubor: 500-550_url.csv
Soubor 500-550_url.csv neexistuje
Ověřuji soubor: 550-600_url.csv
Soubor 550-600_url.csv neexistuje
Ověřuji soubor: 600-613_url.csv
Soubor 600-613_url.csv neexistuje
Ověřování dokončeno!


In [29]:
import pandas as pd
import os

# Nastavení cest
base_dir = "1pracovni/1vysledky/"

# Funkce pro ověření validity vstupního souboru
def validate_input_file(file_path):
    urls = []

    # Zkusíme různé formáty souborů
    try:
        # Zkusíme načíst jako textový soubor (txt, csv)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            # Pokud obsahuje oddělovače, může jít o CSV
            if ',' in content or ';' in content or '\t' in content:
                # Zkusíme různé oddělovače
                for sep in [',', ';', '\t']:
                    try:
                        df = pd.read_csv(file_path, sep=sep, encoding='utf-8')
                        if not df.empty:
                            urls = df.iloc[:, 0].tolist()  # Bereme první sloupec
                            print(f"Soubor úspěšně načten jako CSV s oddělovačem '{sep}'")
                            return True
                    except Exception as e:
                        print(f"Chyba při načítání CSV s oddělovačem '{sep}': {e}")
                        continue

            # Pokud to není CSV, zkusíme to jako prostý textový soubor s URL na každém řádku
            urls = [line.strip() for line in content.split('\n') if line.strip()]
            if urls:
                print("Soubor úspěšně načten jako textový soubor")
                return True
    except Exception as e:
        print(f"Chyba při čtení textového souboru: {e}")
        pass

    # Zkusíme načíst jako Excel
    try:
        df = pd.read_excel(file_path)
        urls = df.iloc[:, 0].tolist()
        print("Soubor úspěšně načten jako Excel")
        return True
    except Exception as e:
        print(f"Chyba při načítání Excel souboru: {e}")
        pass

    # Pokud nic nefunguje, soubor není validní
    return False

# Funkce pro zpracování jednoho souboru
def validate_file(file_name):
    print(f"Ověřuji soubor: {file_name}")
    file_path = os.path.join(base_dir, file_name)
    absolute_path = os.path.abspath(file_path)
    print(f"Absolutní cesta k souboru: {absolute_path}")

    # Ověření existence souboru
    if not os.path.exists(file_path):
        print(f"Soubor {file_name} neexistuje")
        return False

    # Ověření validity souboru
    if not validate_input_file(file_path):
        print(f"Soubor {file_name} není validní")
        return False

    print(f"Soubor {file_name} je validní")
    return True

# Hlavní funkce pro ověření všech souborů
def validate_all_files():
    file_ranges = [
        (100, 150), (150, 200), (200, 250),
        (250, 300), (300, 350), (350, 400), (400, 450), (450, 500),
        (500, 550), (550, 600), (600, 613)
    ]

    for start, end in file_ranges:
        file_name = f"{start}-{end}_url.csv"
        validate_file(file_name)

# Spuštění hlavního procesu
if __name__ == "__main__":
    print("Začínám ověřování vstupních souborů...")
    validate_all_files()
    print("Ověřování dokončeno!")
_

Začínám ověřování vstupních souborů...
Ověřuji soubor: 100-150_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\100-150_url.csv
Soubor 100-150_url.csv neexistuje
Ověřuji soubor: 150-200_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\150-200_url.csv
Soubor 150-200_url.csv neexistuje
Ověřuji soubor: 200-250_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\200-250_url.csv
Soubor 200-250_url.csv neexistuje
Ověřuji soubor: 250-300_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\250-300_url.csv
Soubor 250-300_url.csv neexistuje
Ověřuji soubor: 300-350_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\300-350_url.csv
Soubor 300-350_url.csv neexistuje
Ověřuji soubor: 350-400_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\350-400_url.csv
Soubor 350-400_url.csv neexistuje
Ověřuji soubor: 400-450_url.csv
Absol

'[{"varName": "base_dir", "varType": "str", "varSize": "61", "varShape": "", "varContent": "1pracovni/1vysledky/", "isMatrix": false, "isWidget": false}, {"varName": "files", "varType": "list", "varSize": "56", "varShape": "0", "varContent": "[]", "isMatrix": true, "isWidget": false}, {"varName": "headers", "varType": "dict", "varSize": "184", "varShape": "3 keys", "varContent": "{\'User-Agent\': \'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36\', \'Accept\': \'text/ ...", "isMatrix": false, "isWidget": false}, {"varName": "input_dir", "varType": "str", "varSize": "61", "varShape": "", "varContent": "1pracovni/1vysledky/", "isMatrix": false, "isWidget": false}, {"varName": "output_dir", "varType": "str", "varSize": "68", "varShape": "", "varContent": "1pracovni/1vysledky/scraped", "isMatrix": false, "isWidget": false}]'

In [31]:
import pandas as pd
import os

# Nastavení cest
base_dir = "1pracovni/1vysledky/"

# Funkce pro ověření validity vstupního souboru
def validate_input_file(file_path):
    urls = []

    # Zkusíme různé formáty souborů
    try:
        # Zkusíme načíst jako textový soubor (txt, csv)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            # Pokud obsahuje oddělovače, může jít o CSV
            if ',' in content or ';' in content or '\t' in content:
                # Zkusíme různé oddělovače
                for sep in [',', ';', '\t']:
                    try:
                        df = pd.read_csv(file_path, sep=sep, encoding='utf-8')
                        if not df.empty:
                            urls = df.iloc[:, 0].tolist()  # Bereme první sloupec
                            print(f"Soubor úspěšně načten jako CSV s oddělovačem '{sep}'")
                            return True
                    except Exception as e:
                        print(f"Chyba při načítání CSV s oddělovačem '{sep}': {e}")
                        continue

            # Pokud to není CSV, zkusíme to jako prostý textový soubor s URL na každém řádku
            urls = [line.strip() for line in content.split('\n') if line.strip()]
            if urls:
                print("Soubor úspěšně načten jako textový soubor")
                return True
    except Exception as e:
        print(f"Chyba při čtení textového souboru: {e}")
        pass

    # Zkusíme načíst jako Excel
    try:
        df = pd.read_excel(file_path)
        urls = df.iloc[:, 0].tolist()
        print("Soubor úspěšně načten jako Excel")
        return True
    except Exception as e:
        print(f"Chyba při načítání Excel souboru: {e}")
        pass

    # Pokud nic nefunguje, soubor není validní
    return False

# Funkce pro zpracování jednoho souboru
def validate_file(file_name):
    print(f"Ověřuji soubor: {file_name}")
    file_path = os.path.join(base_dir, file_name)
    absolute_path = os.path.abspath(file_path)
    print(f"Absolutní cesta k souboru: {absolute_path}")

    # Ověření existence souboru
    if not os.path.exists(file_path):
        print(f"Soubor {file_name} neexistuje")
        return False

    # Ověření validity souboru
    if not validate_input_file(file_path):
        print(f"Soubor {file_name} není validní")
        return False

    print(f"Soubor {file_name} je validní")
    return True

# Hlavní funkce pro ověření všech souborů
def validate_all_files():
    file_ranges = [
        (100, 150), (150, 200), (200, 250),
        (250, 300), (300, 350), (350, 400), (400, 450), (450, 500),
        (500, 550), (550, 600), (600, 613)
    ]

    for start, end in file_ranges:
        file_name = f"{start}-{end}_url.csv"
        validate_file(file_name)

# Spuštění hlavního procesu
if __name__ == "__main__":
    print("Začínám ověřování vstupních souborů...")
    validate_all_files()
    print("Ověřování dokončeno!")


Začínám ověřování vstupních souborů...
Ověřuji soubor: 100-150_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\100-150_url.csv
Soubor 100-150_url.csv neexistuje
Ověřuji soubor: 150-200_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\150-200_url.csv
Soubor 150-200_url.csv neexistuje
Ověřuji soubor: 200-250_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\200-250_url.csv
Soubor 200-250_url.csv neexistuje
Ověřuji soubor: 250-300_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\250-300_url.csv
Soubor 250-300_url.csv neexistuje
Ověřuji soubor: 300-350_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\300-350_url.csv
Soubor 300-350_url.csv neexistuje
Ověřuji soubor: 350-400_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\350-400_url.csv
Soubor 350-400_url.csv neexistuje
Ověřuji soubor: 400-450_url.csv
Absol

In [33]:
import pandas as pd
import os

# Nastavení cest
base_dir = "1pracovni/1vysledky/"

# Funkce pro ověření validity vstupního souboru
def validate_input_file(file_path):
    urls = []

    # Zkusíme různé formáty souborů
    try:
        # Zkusíme načíst jako textový soubor (txt, csv)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            # Pokud obsahuje oddělovače, může jít o CSV
            if ',' in content or ';' in content or '\t' in content:
                # Zkusíme různé oddělovače
                for sep in [',', ';', '\t']:
                    try:
                        df = pd.read_csv(file_path, sep=sep, encoding='utf-8')
                        if not df.empty:
                            urls = df.iloc[:, 0].tolist()  # Bereme první sloupec
                            print(f"Soubor úspěšně načten jako CSV s oddělovačem '{sep}'")
                            return True
                    except Exception as e:
                        print(f"Chyba při načítání CSV s oddělovačem '{sep}': {e}")
                        continue

            # Pokud to není CSV, zkusíme to jako prostý textový soubor s URL na každém řádku
            urls = [line.strip() for line in content.split('\n') if line.strip()]
            if urls:
                print("Soubor úspěšně načten jako textový soubor")
                return True
    except Exception as e:
        print(f"Chyba při čtení textového souboru: {e}")
        pass

    # Zkusíme načíst jako Excel
    try:
        df = pd.read_excel(file_path)
        urls = df.iloc[:, 0].tolist()
        print("Soubor úspěšně načten jako Excel")
        return True
    except Exception as e:
        print(f"Chyba při načítání Excel souboru: {e}")
        pass

    # Pokud nic nefunguje, soubor není validní
    return False

# Funkce pro zpracování jednoho souboru
def validate_file(file_name):
    print(f"Ověřuji soubor: {file_name}")
    file_path = os.path.join(base_dir, file_name)
    absolute_path = os.path.abspath(file_path)
    print(f"Absolutní cesta k souboru: {absolute_path}")

    # Ověření existence souboru
    if not os.path.exists(file_path):
        print(f"Soubor {file_name} neexistuje")
        return False

    # Ověření validity souboru
    if not validate_input_file(file_path):
        print(f"Soubor {file_name} není validní")
        return False

    print(f"Soubor {file_name} je validní")
    return True

# Hlavní funkce pro ověření všech souborů
def validate_all_files():
    file_ranges = [
        (100, 150), (150, 200), (200, 250),
        (250, 300), (300, 350), (350, 400), (400, 450), (450, 500),
        (500, 550), (550, 600), (600, 613)
    ]

    for start, end in file_ranges:
        file_name = f"{start}-{end}_url.csv"
        validate_file(file_name)

# Spuštění hlavního procesu
if __name__ == "__main__":
    print("Začínám ověřování vstupních souborů...")
    validate_all_files()
    print("Ověřování dokončeno!")



Začínám ověřování vstupních souborů...
Ověřuji soubor: 100-150_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\100-150_url.csv
Soubor 100-150_url.csv neexistuje
Ověřuji soubor: 150-200_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\150-200_url.csv
Soubor 150-200_url.csv neexistuje
Ověřuji soubor: 200-250_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\200-250_url.csv
Soubor 200-250_url.csv neexistuje
Ověřuji soubor: 250-300_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\250-300_url.csv
Soubor 250-300_url.csv neexistuje
Ověřuji soubor: 300-350_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\300-350_url.csv
Soubor 300-350_url.csv neexistuje
Ověřuji soubor: 350-400_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\350-400_url.csv
Soubor 350-400_url.csv neexistuje
Ověřuji soubor: 400-450_url.csv
Absol

In [35]:
import pandas as pd
import os

# Nastavení cest
base_dir = "1pracovni/1vysledky/"

# Funkce pro ověření validity vstupního souboru
def validate_input_file(file_path):
    urls = []

    # Zkusíme různé formáty souborů
    try:
        # Zkusíme načíst jako textový soubor (txt, csv)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            # Pokud obsahuje oddělovače, může jít o CSV
            if ',' in content or ';' in content or '\t' in content:
                # Zkusíme různé oddělovače
                for sep in [',', ';', '\t']:
                    try:
                        df = pd.read_csv(file_path, sep=sep, encoding='utf-8')
                        if not df.empty:
                            urls = df.iloc[:, 0].tolist()  # Bereme první sloupec
                            print(f"Soubor úspěšně načten jako CSV s oddělovačem '{sep}'")
                            return True
                    except Exception as e:
                        print(f"Chyba při načítání CSV s oddělovačem '{sep}': {e}")
                        continue

            # Pokud to není CSV, zkusíme to jako prostý textový soubor s URL na každém řádku
            urls = [line.strip() for line in content.split('\n') if line.strip()]
            if urls:
                print("Soubor úspěšně načten jako textový soubor")
                return True
    except Exception as e:
        print(f"Chyba při čtení textového souboru: {e}")
        pass

    # Zkusíme načíst jako Excel
    try:
        df = pd.read_excel(file_path)
        urls = df.iloc[:, 0].tolist()
        print("Soubor úspěšně načten jako Excel")
        return True
    except Exception as e:
        print(f"Chyba při načítání Excel souboru: {e}")
        pass

    # Pokud nic nefunguje, soubor není validní
    return False

# Funkce pro zpracování jednoho souboru
def validate_file(file_name):
    print(f"Ověřuji soubor: {file_name}")
    file_path = os.path.join(base_dir, file_name)
    absolute_path = os.path.abspath(file_path)
    print(f"Absolutní cesta k souboru: {absolute_path}")

    # Ověření existence souboru
    if not os.path.exists(file_path):
        print(f"Soubor {file_name} neexistuje")
        return False

    # Ověření validity souboru
    if not validate_input_file(file_path):
        print(f"Soubor {file_name} není validní")
        return False

    print(f"Soubor {file_name} je validní")
    return True

# Hlavní funkce pro ověření všech souborů
def validate_all_files():
    file_ranges = [
        (100, 150), (150, 200), (200, 250),
        (250, 300), (300, 350), (350, 400), (400, 450), (450, 500),
        (500, 550), (550, 600), (600, 613)
    ]

    for start, end in file_ranges:
        file_name = f"{start}-{end}_url.csv"
        validate_file(file_name)

# Spuštění hlavního procesu
if __name__ == "__main__":
    print("Začínám ověřování vstupních souborů...")
    validate_all_files()
    print("Ověřování dokončeno!")


Začínám ověřování vstupních souborů...
Ověřuji soubor: 100-150_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\100-150_url.csv
Soubor 100-150_url.csv neexistuje
Ověřuji soubor: 150-200_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\150-200_url.csv
Soubor 150-200_url.csv neexistuje
Ověřuji soubor: 200-250_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\200-250_url.csv
Soubor 200-250_url.csv neexistuje
Ověřuji soubor: 250-300_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\250-300_url.csv
Soubor 250-300_url.csv neexistuje
Ověřuji soubor: 300-350_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\300-350_url.csv
Soubor 300-350_url.csv neexistuje
Ověřuji soubor: 350-400_url.csv
Absolutní cesta k souboru: C:\Users\miros\1pracovni\1pracovni\1vysledky\350-400_url.csv
Soubor 350-400_url.csv neexistuje
Ověřuji soubor: 400-450_url.csv
Absol